In [ ]:
import numpy as np
import minterpy as mp
import minterpy_levelsets as ls

In [ ]:

# Setting up the initial configuration
N_points = 400
ic_points = ls.points_on_biconcave_disc(N_points, param_c=0.5, param_d=0.375, random_seed=42)
# ic_points = ls.points_on_ellipsoid(N_points, 0.8, 0.9, 1.0)

max_steps = 100
output_freq = 1
step = 0
curr_points = ic_points
old_points = ic_points


In [ ]:

print(f"#Step\tDt\tDegree\tlevel_dim\tregression_error\trecalibration_error")

# Main loop of the mean curvature flow problem
while step < max_steps:
    if step == 0:
        # For the initial configuration, use a good fit
        poly = ls.LevelsetPoly(curr_points, method='BK', tol=1e-10)
    else:
        poly = ls.LevelsetPoly(curr_points, method='BK', tol=1e-4)

        
    new_points = poly.closest_point_projection(curr_points, tol=1e-4)
    
    curr_points = new_points.copy()
    
    #Compute gradients and curvatures from the BKs
    grad_vals = poly.compute_gradients_at(curr_points)
    curvatures,_ = poly.compute_curvatures_at(curr_points)
    
    #Normalize the gradient before computing 
    grad_vals_norm = grad_vals / np.linalg.norm(grad_vals,2,axis=1)[:,None]
    
    velocities = curvatures[:,None] * grad_vals_norm
    
    # Generate output
    if step % output_freq == 0:
        poly.output_VTR(frame=int(step / output_freq), mesh_size=50)
        
        ls.output_VTK(curr_points, frame=int(step / output_freq), vector_field=grad_vals)
 
    old_points = curr_points.copy()
    dt = 2e-3/np.max(np.abs(curvatures))
    
    new_points = curr_points + dt * velocities
    
    curr_points = new_points
    level_dim = 1
    
    poly_degree = poly._newton_poly.multi_index.poly_degree
    print(f"{step}\t{dt}\t{poly_degree}")
    step += 1